# NewsBERT on Colab

This notebook uses awesome [streamlit tutorial](https://youtu.be/x0NdZkaciws) to run NewsBERT on Colab.

For running this code you will need to setup your [ngrok](https://ngrok.com/) account

In [ ]:
!pip install streamlit pyngrok
!pip install git+https://github.com/lambdaofgod/pytorch_hackathon/

In [ ]:
!pip install torch==1.5.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4MB 24kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.5.1+cu101 which is incompatible.
  Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import tqdm
import os
from operator import itemgetter

import torch
from pytorch_hackathon import rss_feeds, zero_shot_learning, haystack_search
import seaborn as sns

st.title('Zero-shot RSS feed article classifier')

cm = sns.light_palette("green", as_cmap=True)
topic_strings = list(pd.read_table('https://raw.githubusercontent.com/lambdaofgod/pytorch_hackathon/master/data/topics.txt', header=None).iloc[:,0].values)
rss_feed_urls = list(pd.read_table('https://raw.githubusercontent.com/lambdaofgod/pytorch_hackathon/master/data/feeds.txt', header=None).iloc[:,0].values)
rss_feed_urls = rss_feeds.rss_feed_urls.copy()


model_device = st.selectbox("Model device", ["cpu", "cuda"], index=int(torch.cuda.is_available()))


@st.cache(allow_output_mutation=True)
def get_feed_df():
    with st.spinner('Retrieving articles from feeds...'):
        return rss_feeds.get_feed_df(rss_feed_urls)


feed_df = get_feed_df()


@st.cache(allow_output_mutation=True)
def setup_searcher(feed_df, use_gpu, model_name="deepset/sentence_bert"):
    with st.spinner('No precomputed topics found, running zero-shot learning...'):
        searcher = haystack_search.Searcher(model_name, 'text', use_gpu=use_gpu)
        searcher.add_texts(feed_df)
    return searcher 


# we need to copy feed_df so that streamlit doesn't recompute embeddings when feed_df changes 
searcher = setup_searcher(feed_df, use_gpu=model_device == 'cuda') 


@st.cache
def get_retrieved_df(topic_strings):
    results = [
        result 
        for topic in topic_strings
        for result in searcher.retriever.retrieve(
            "text is about {}".format(topic)
        )
    ]
    return searcher.get_topic_score_df(
        results,
        topic_strings
    ).drop_duplicates(subset='title')
    

selected_df = get_retrieved_df(topic_strings).reset_index(drop=True)
selected_df['text'] = selected_df['text'].apply(lambda s: s[:1000])
topics = st.multiselect('Choose topics', topic_strings, default=[topic_strings[0]])
sort_by = st.selectbox("Sort by", topics)
display_df = selected_df[selected_df[topics].min(axis=1) > 0.5].sort_values(sort_by, ascending=False)

st.markdown('## Articles on {}'.format(', '.join(topics)))

st.table(display_df[display_df[topics].min(axis=1) > 0.5].style.background_gradient(cmap=cm))

Overwriting app.py


In [ ]:
## Checking if GPU is available

Running this on GPU will be much faster (embeddings for articles will be calculated much faster).

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from pyngrok import ngrok

# ngrok token

You will need to paste your ngrok authtoken. You can find it [here](https://dashboard.ngrok.com/auth/your-authtoken) provided you have an ngrok account.

In [ ]:
#@title ngrok token

token = '' #@param {type:"string"}
!ngrok authtoken $token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!streamlit run app.py&>log&

In [ ]:
!cat log


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8503
  External URL: http://34.87.86.86:8503

2020-08-16 18:02:32.817699: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
100%|██████████| 16/16 [00:13<00:00,  1.17it/s]
/usr/local/lib/python3.6/dist-packages/pytorch_hackathon/rss_feeds.py:63: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 63 of the file /usr/local/lib/python3.6/dis

In [ ]:
publ_url = ngrok.connect(port='8503')

In [ ]:
publ_url

'http://635eda040288.ngrok.io'